# Analysing the GDELT events

In [1]:
#!pip install gdelt

In [1]:
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
import datetime as dt
import gdelt as gd
import numpy as np
import pandas as pd

In [2]:
gdelt_client = gd.gdelt()

In [3]:
date = dt.datetime.now() - dt.timedelta(days=1)
date_string = date.strftime('%Y %m %d')
gdelt_events = gdelt_client.Search(date_string, table='events', coverage=True)

In [4]:
gdelt_events['CAMEOCodeDescription'].unique()
gdelt_events.columns

Index(['GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate',
       'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
       'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
       'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code',
       'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code',
       'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent',
       'EventCode', 'CAMEOCodeDescription', 'EventBaseCode', 'EventRootCode',
       'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources',
       'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_FullName',
       'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_ADM2Code',
       'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID',
       'Actor2Geo_Type', 'Actor2Geo_FullName', 'Actor2Geo_CountryCode',
       'Actor2Geo_ADM1Code', 'Actor2Geo_ADM2Code

In [5]:
#gdelt_events.groupby(['Actor1Name', 'ActionGeo_FullName', 'Actor2Name', 'CAMEOCodeDescription']).size().sort_values(ascending=False).reset_index(name='counts')
gdelt_events.groupby(['ActionGeo_FullName', 'CAMEOCodeDescription']).size().sort_values(ascending=False).reset_index(name='counts')

,ActionGeo_FullName,CAMEOCodeDescription,counts
0,"Port-Au-Prince, Ouest, Haiti","Make statement, not specified below",116
1,"Port-Au-Prince, Ouest, Haiti",Make a visit,105
2,"Washington, District of Columbia, United States","Make statement, not specified below",105
3,United States,"Make statement, not specified below",103
4,"Port-Au-Prince, Ouest, Haiti",Host a visit,102
...,...,...,...
23291,"Marsa Alam, Al Ba?r al A?mar, Egypt","Appeal, not specified below",1
23292,"Mark Twain, Missouri, United States",Make a visit,1
23293,"Marion County, Ohio, United States","Make statement, not specified below",1
23294,"Marion County, Iowa, United States",Make a visit,1


In [7]:
# Connect to ArcGIS Online anonymously
gis = GIS()

europe_map = gis.map("Europe")
europe_map.basemap = "dark-gray-vector"

In [8]:
WGS84 = 4326
gdelt_spatial = GeoAccessor.from_xy(gdelt_events, x_column='ActionGeo_Long', y_column='ActionGeo_Lat', sr=WGS84)
gdelt_spatial.spatial.plot(map_widget=europe_map, renderer_type='s')
europe_map

MapView(layout=Layout(height='400px', width='100%'))

In [9]:
import algorithm

In [10]:
gdelt_events.groupby(['ActionGeo_FullName']).size().sort_values(ascending=False).reset_index(name='counts')

,ActionGeo_FullName,counts
0,"Gaza, Israel (general), Israel",4456
1,"Jerusalem, Israel (general), Israel",2409
2,United States,1359
3,"Washington, District of Columbia, United States",1160
4,"New York, United States",1095
...,...,...
4973,"Kill Devil Hills, North Carolina, United States",1
4974,"King Saud University, Ar Riya?, Saudi Arabia",1
4975,"Kingdom Of Buganda, Mityana, Uganda",1
4976,"Kingstown, Saint George, Saint Vincent And The...",1


In [26]:
names_values = gdelt_events['ActionGeo_FullName'].dropna()
names_list = names_values.tolist()
algorithm.count(names_list, 'Gaza, Israel (general), Israel')

4456

In [27]:
names_arr = names_values.to_numpy()
algorithm.count(names_arr, 'Gaza, Israel (general), Israel')

4456